# Load Data

In [1]:
from LoadData import outcomes_df, projects_df, donors_choose_df

In [2]:
import pandas as pd
import numpy as np
# TO DEBUG:  import pdb

In [3]:
import ExplorationFunctions as ef
import LoadData as ld
from Variables import *
import PipelineFunctions as pf

/Users/akoko0530/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.

/Users/akoko0530/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.



In [4]:
remove_from_projects = GEO_VARIABLES + ID_VARIABLES + IDX
keep_vars = [x for x in projects_df.columns if x not in remove_from_projects]
labeled_df = donors_choose_df[keep_vars + TARGET_VARIABLES]
# labeled_df['date_posted'] = pd.to_datetime(labeled_df['date_posted'])
    # with more time, would want to get temporal splits funct to work using datetime module
labeled_df = ef.change_to_1_0(labeled_df)
labeled_df = ef.impute_mean(labeled_df, given_cols=CONTINUOUS_VARIABLES)

/Users/akoko0530/Documents/GitHub/MachineLearning_CAPP30254/Homework_3/ExplorationFunctions.py:83: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/akoko0530/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [5]:
#remove outliers
for var in CONTINUOUS_VARIABLES:
    labeled_df = ef.zscore_remove_outlier(labeled_df, var)
#discretize
for var in CONTINUOUS_VARIABLES:
    labeled_df = ef.discretize(labeled_df, var, 10, want_quantile=False)

In [6]:
dum_variables_of_interest = [
'poverty_level', 
'teacher_prefix',
'primary_focus_area']

In [7]:
labeled_df = ef.dummytize(labeled_df, dum_variables_of_interest)

In [8]:
features = ['school_nlns', 'school_kipp',
            'fulfillment_labor_materials',
            'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
            'high poverty', 'highest poverty', 'low poverty','moderate poverty', 
            'Dr.', 'Mr.', 'Mrs.', 'Ms.', 
            'Applied Learning','Health & Sports', 'History & Civics', 
            'Literacy & Language','Math & Science', 'Music & The Arts', 'Special Needs']
target = TARGET_VARIABLES

In [9]:
labeled_df = labeled_df[features + CONTINUOUS_VARIABLES + DATE_VARIABLE + target]

In [10]:
ef.has_na(labeled_df)

[]

no rows to drop

# Simple Loop & Temporal Validation

In [11]:
from __future__ import division
from sklearn import preprocessing, cross_validation, svm, metrics, tree, decomposition, svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier, OrthogonalMatchingPursuit, RandomizedLogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import ParameterGrid
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
import random
import matplotlib.pyplot as plt
from scipy import optimize
import time
import seaborn as sns
%matplotlib inline

# from sklearn.metrics import precision_recall_curve


### First Temporal Split:

- Train: ('2011-01-01', '2011-12-31')
- Test: ('2012-01-01', '2012-06-31')

In [12]:
X_train, X_test, y_train, y_test = pf.temporal_splitter(labeled_df, 
                                                                features, target, 1)

x_train min: 2011-01-01 , x_train max:  2011-12-31
y_train min: 2011-01-01 , y_train max:  2011-12-31
x_test min: 2012-01-01 , x_test max:  2012-06-30
y_test min: 2012-01-01 , y_test max:  2012-06-30


In [13]:
pf.go_function(X_train, X_test, y_train, y_test['fully_funded'])

RF


ValueError: cannot set a row with mismatched columns

### Second Temporal Split:

- Train: ('2011-01-01', '2012-06-31')
- Test: ('2012-07-01', '2012-12-31')

In [ ]:
X_train, X_test, y_train, y_test = pf.temporal_splitter(labeled_df, 
                                                                features, target, 2)

In [ ]:
pf.go_function(X_train, X_test, y_train, y_test['fully_funded'])

### Third Temporal Split:

- Train: ('2011-01-01', '2012-12-31')
- Test: ('2013-01-01', '2013-06-31')

In [ ]:
X_train, X_test, y_train, y_test = pf.temporal_splitter(labeled_df, 
                                                                features, target, 3)

In [ ]:
pf.go_function(X_train, X_test, y_train, y_test['fully_funded'])

### Fourth Temporal Split:

- Train: ('2011-01-01', '2013-06-31')
- Test: ('2013-07-01', '2013-12-31')

In [ ]:
X_train, X_test, y_train, y_test = pf.temporal_splitter(labeled_df, 
                                                                features, target, 4)

In [ ]:
pf.go_function(X_train, X_test, y_train, y_test['fully_funded'])